In [172]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time
import requests
import time as t
from commonregex import CommonRegex
from streetaddress import StreetAddressFormatter, StreetAddressParser
import pandas as pd

In [32]:
!which chromedriver

/opt/homebrew/bin/chromedriver


In [34]:
executable_path = {'executable_path': '/opt/homebrew/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [26]:
State = "ca" + "/"

City = "palm-desert"

st_city_url = url + State + City
st_city_url

'https://www.golflink.com/golf-courses/ca/palm-desert'

In [35]:
browser.visit("https://www.golflink.com/golf-courses/ca/palm-springs/")

In [36]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [170]:
# all_divs = []
# for course in soup.find_all("dic", class_="item", href=True):
course_dict = {}
dict_list = []

courses = soup.find_all("div", class_="resultContainer")
for course in courses:
    course_dict = {}
    name = course.find("a", class_="fly")
    address_blurb = course.find("div", class_ = "textcontainer")
#     print(address_blurb.text)
    text_add_blur = address_blurb.text
    clean_add_blurb = " ".join(text_add_blur.split())
    course_dict["Name"] = name.text
    course_dict["Holes"] = re.search(r'.*holes', clean_add_blurb)[0]
    try:
        pub_priv = re.findall('(Private|Public|Municipal)', clean_add_blurb)[0]
        course_dict["Public/Private"] = pub_priv
    except:
        course_dict["Public/Private"] = "N/A"
    course_dict["Phone"] = re.findall("\(\w{3}\) \w{3}-\w{4}",clean_add_blurb)[0]

    phone = re.findall("\(\w{3}\) \w{3}-\w{4}",clean_add_blurb)
    add = clean_add_blurb.rpartition(phone[0])[0]
    
    course_dict["Address"] = (re.split('Write a Review Check Tee Times |Write a Review ', add)[-1])
    
    course_dict["Blurb"] = clean_add_blurb
#     word = clean_add_blurb[0]
#     print(clean_add_blurb)
#     before_keyword, keyword, after_keyword = word.partition(phone)
#     print(f"Before:{before_keyword}")
#     print(f"mid: {keyword}")
#     print(f"After: {after_keyword}")
#     # initializing substrings
#     sub1 = "is"
#     sub2 = "and"
  
#     # getting index of substrings
#     idx1 = test_str.index(sub1)
#     idx2 = test_str.index(phone)
  
#     # length of substring 1 is added to
#     # get string from next character
#     res = test_str[idx1 + len(sub1) + 1: idx2]
#     print(address)
#     parse = CommonRegex(clean_add_blurb)
#     print(parse.street_addresses)
#     addr_parser = StreetAddressParser()

#     addr = addr_parser.parse(clean_add_blurb)
#     address = re.findall("\d+[ ](?:[A-Za-z0-9.-]+[ ]?)+(?:Avenue|Lane|Road|Boulevard|Drive|Street|Ave|Dr|Rd|Blvd|Ln|St)\.?", clean_add_blurb)
#     print(address)
    dict_list.append(course_dict)


In [171]:
dict_list

[{'Name': "O'Donnell Golf Club, O'Donnell Course",
  'Holes': '9 holes',
  'Public/Private': 'Private',
  'Phone': '(760) 325-2259',
  'Address': '301 N Belardo Rd Palm Springs, CA 92262-5600 ',
  'Blurb': '9 holes over 2,655 yards with a par of 35 (Private) Write a Review 301 N Belardo Rd Palm Springs, CA 92262-5600 (760) 325-2259 0 miles from the center of Palm Springs. Write a Review'},
 {'Name': 'Mesquite Golf and Country Club, Mesquite Course',
  'Holes': '18 holes',
  'Public/Private': 'Public',
  'Phone': '(760) 992-5158',
  'Address': '2700 E Mesquite Ave Ofc Palm Springs, CA 92264-5009 ',
  'Blurb': '18 holes over 6,328 yards with a par of 72 (Public) Write a Review Check Tee Times 2700 E Mesquite Ave Ofc Palm Springs, CA 92264-5009 (760) 992-5158 2 miles from the center of Palm Springs. Write a Review Check Tee Times'},
 {'Name': 'Escena Golf Club, Escena Course',
  'Holes': '18 holes',
  'Public/Private': 'Public',
  'Phone': '(760) 778-2737',
  'Address': '1100 Clubhouse Vi

In [173]:
df = pd.DataFrame.from_dict(dict_list)
df

,Name,Holes,Public/Private,Phone,Address,Blurb
0,"O'Donnell Golf Club, O'Donnell Course",9 holes,Private,(760) 325-2259,"301 N Belardo Rd Palm Springs, CA 92262-5600","9 holes over 2,655 yards with a par of 35 (Pri..."
1,"Mesquite Golf and Country Club, Mesquite Course",18 holes,Public,(760) 992-5158,"2700 E Mesquite Ave Ofc Palm Springs, CA 92264...","18 holes over 6,328 yards with a par of 72 (Pu..."
2,"Escena Golf Club, Escena Course",18 holes,Public,(760) 778-2737,"1100 Clubhouse View Dr Palm Springs, CA 92262-...","18 holes over 7,211 yards with a par of 72 (Pu..."
3,"Canyon Estates, Canyon Estates Course",9 holes,Private,(760) 327-1346,"2323 S Madrona Dr Palm Springs, CA 92264-9514",9 holes (Private) Write a Review 2323 S Madron...
4,"Seven Lakes Country Club, Seven Lakes Course",18 holes,Private,(760) 328-9774,"1 Desert Lakes Dr Palm Springs, CA 92264-5520","18 holes over 2,712 yards with a par of 58 (Pr..."
...,...,...,...,...,...,...
95,"Bermuda Dunes Country Club, The Lake Course",9 holes,Private,(760) 360-2481,"42765 Adams St Bermuda Dunes, CA 92203-7937","9 holes over 3,269 yards with a par of 36 (Pri..."
96,"PGA West Mountain & Dunes Golf Courses, Pete D...",18 holes,Public,(760) 564-7610,"50200 Avenida Vista Bonita La Quinta, CA 92253...","18 holes over 6,756 yards with a par of 72 (Pu..."
97,"PGA West Mountain & Dunes Golf Courses, Pete D...",18 holes,Public,(760) 564-7610,"50200 Avenida Vista Bonita La Quinta, CA 92253...","18 holes over 6,074 yards with a par of 72 (Pu..."
98,"La Quinta Country Club, La Quinta Course",18 holes,Private,(760) 564-4151,"77750 Avenue 50 La Quinta, CA 92253-2204","18 holes over 7,060 yards with a par of 72 (Pr..."
